# Seach fastq for sequences
This notebook will help you determine if certain sequences exist in your fastq file.

### Usage
Please provide file paths for a fastq file and a list of sequences

In [ ]:
# Run if dnaio is not installed
!pip install dnaio

In [53]:
### INPUTS ###
fastq_file_path = "data/FX4-230125-02_S4_L001_R2_001.fastq.gz"
sequence_list_path = "data/list_of_sequences.csv"
output_file_path = "data/FX4-230125-02_S4_L001_R2_001.output.not_parallel.csv"
njobs = None
reverse = False

In [54]:
import dnaio
import csv
from joblib import Parallel, delayed

In [55]:
'''Read and store list of sequences'''
file = open(sequence_list_path, "r")
sequence_list = file.read().split("\n")
file.close()

# Will only run if you would like to search for the reverse of your sequences
if reverse:
    reverse_sequences = []
    for sequence in sequence_list:
        reverse_sequences.append(sequence[::-1])
    sequence_list = reverse_sequences

In [56]:
def search_reads(read_number: int, read_sequence: str, sequence_list: list, output_file_path: str):
    ''' Search a read sequence for sequences in list
    Meant to be run in parallel
    '''
    for sequence in sequence_list:
        if sequence in read_sequence:
            # Open output file to append
            output_file = open(output_file_path, "a")
            output_file.write(f"{read_number},{read_sequence},{sequence},{read_sequence.find(sequence)}\n")
            output_file.close()
            break #In theory there should only be one sequence per read

In [57]:
'''Search fastq file for sequences'''

# Open output file and clear
output_file = open(output_file_path, "w")
output_file.write("read_number,read_sequence,pattern_found,nucleotide_index\n")
output_file.close()

# Set njobs if not set
njobs = -1 if not njobs else njobs

# Open fastq and run in parallel
with dnaio.open(fastq_file_path) as f:
    Parallel(n_jobs=njobs, backend="multiprocessing")(delayed(search_reads)(read_number=i, read_sequence=read.sequence, sequence_list=sequence_list, output_file_path=output_file_path) for i, read in enumerate(f))
    